In [8]:
import pymssql
import pathlib
import pydicom
import os
import pathlib
import numpy as np
import h5py
import xml.etree.ElementTree as ET
from scipy.io import savemat
import pandas as pd
import math
from tqdm import tqdm, trange


In [9]:
Root = 'D:\LDCT_TEST\part1'
patientID = os.listdir(Root)

In [10]:
connect1 = pymssql.connect(
    host='172.31.6.157',
    user='TEST',
    password='81218',
    database='AICH'
)
connect2 = pymssql.connect(
    host='172.31.6.157',
    user='TEST',
    password='81218',
    database='AICH'
)
connect3 = pymssql.connect(
    host='172.31.6.157',
    user='TEST',
    password='81218',
    database='AICH'
)
PatientInfo_cursor = connect1.cursor(as_dict=True)
LDCTInfo_cursor = connect2.cursor(as_dict=True)
Annotation_cursor = connect3.cursor(as_dict=True)

In [11]:
query_search_PatientInfo = '''
    select * from LDCT_PatientInfo where PartiID in (
        select distinct seriesDes from ExamStudySeries_6 as a
        inner join allEvents as b on a.eventID=b.eventID
        where chartNo=%(chartNo)s
    )
'''

In [12]:
query_search_LDCT_info = '''
    select * from LDCT_info as a
    inner join LDCT_Manufacturer as b on a.ManuID=b.ManuID
    where PartiID in (
        select distinct seriesDes from ExamStudySeries_6 as a
        inner join allEvents as b on a.eventID=b.eventID
        where chartNo=%(chartNo)s
    )
'''

In [13]:
query_search_annotation = '''
    select * from(
        select * from Localization where Disease=5
    ) as a
    inner join(
        select a.*,b.chartNo from ExamStudySeries_6 as a 
        inner join allEvents as b on a.eventID=b.eventID
    )as b 
    on a.PID=b.chartNo and a.StudyID=b.studyID and a.seriesID=b.seriesID and a.SD=b.studyDate
    where PID = %(chartNo)s
'''

In [14]:
for chartNo in patientID:
    for dirPath, dirNames, fileNames in os.walk(os.path.join(Root,chartNo)):
        Dir=dirPath
    '''-----------------------------------------------------------------------'''
    PatientInfo_cursor.execute(query_search_PatientInfo, {'chartNo': chartNo})
    LDCTInfo_cursor.execute(query_search_LDCT_info, {'chartNo': chartNo})
    Annotation_cursor.execute(query_search_annotation, {'chartNo': chartNo})
    Annotation_res = Annotation_cursor.fetchall()
    PatientInfo_res = PatientInfo_cursor.fetchall()
    LDCTInfo_cursor_res = LDCTInfo_cursor.fetchall()
    if len(Annotation_res) > 0:
        '''-----------------------------------------------------------------------'''
        Main = ET.Element('Annotation')
        HospitalInfo = ET.SubElement(Main, 'HospitalInfo')
        Hospital = ET.SubElement(HospitalInfo, 'Hospital')
        Hospital.text = '06'
        '''-----------------------------------------------------------------------'''
        PatientInfo = ET.SubElement(Main, 'PatientInfo')
        PatientID = ET.SubElement(PatientInfo, 'PatientID')
        PatientID.text = 'Anonymous'
        ParticipantID = ET.SubElement(PatientInfo, 'ParticipantID')
        ParticipantID.text = str(PatientInfo_res[0]['PartiID'])
        LDCTDate = ET.SubElement(PatientInfo, 'LDCTDate')
        LDCTDate.text = str(LDCTInfo_cursor_res[0]['LDCTDate'])
        SeriesNumber = ET.SubElement(PatientInfo, 'SeriesNumber')
        SeriesNumber.text = str(LDCTInfo_cursor_res[0]['SeriesNumber'])
        Age = ET.SubElement(PatientInfo, 'Age')
        Age.text = str(PatientInfo_res[0]['Age'])
        Sex = ET.SubElement(PatientInfo, 'Sex')
        Sex.text = str(PatientInfo_res[0]['Sex'])
        BirthDate = ET.SubElement(PatientInfo, 'BirthDate')
        BirthDate.text = str(PatientInfo_res[0]['BirthDate'])
        Height = ET.SubElement(PatientInfo, 'Height')
        Height.text = str(PatientInfo_res[0]['Height'])
        Weight = ET.SubElement(PatientInfo, 'Weight')
        Weight.text = str(PatientInfo_res[0]['Weight'])
        '''-----------------------------------------------------------------------'''
        LDCTInfo = ET.SubElement(Main, 'LDCTInfo')
        TotalDLP = ET.SubElement(LDCTInfo, 'TotalDLP')
        TotalDLP.text = str(LDCTInfo_cursor_res[0]['TotalDLP'])
        CTDIvol = ET.SubElement(LDCTInfo, 'CTDIvol')
        CTDIvol.text = str(LDCTInfo_cursor_res[0]['CTDIvol'])
        InstitutionName = ET.SubElement(LDCTInfo, 'InstitutionName')
        InstitutionName.text = str(LDCTInfo_cursor_res[0]['Institution'])
        ImageType = ET.SubElement(LDCTInfo, 'ImageType')
        ImageType.text = str(LDCTInfo_cursor_res[0]['ImageType'])
        Manufacturer = ET.SubElement(LDCTInfo, 'Manufacturer')
        Manufacturer.text = str(LDCTInfo_cursor_res[0]['ManuModel'])
        ScannerModel = ET.SubElement(LDCTInfo, 'ScannerModel')
        ScannerModel.text = str(LDCTInfo_cursor_res[0]['Manufacturer'])
        DataCollectionDiameter = ET.SubElement(LDCTInfo, 'DataCollectionDiameter')
        DataCollectionDiameter.text = str(LDCTInfo_cursor_res[0]['DataColDia'])
        ReconDiameter = ET.SubElement(LDCTInfo, 'ReconDiameter')
        ReconDiameter.text = str(LDCTInfo_cursor_res[0]['ReconDia'])
        Exposure = ET.SubElement(LDCTInfo, 'Exposure')
        Exposure.text = str(LDCTInfo_cursor_res[0]['Exposure'])
        XRayTubeCurrent = ET.SubElement(LDCTInfo, 'XRayTubeCurrent')
        XRayTubeCurrent.set('Max',str(LDCTInfo_cursor_res[0]['MaxTubeCurr']))
        XRayTubeCurrent.set('Min',str(LDCTInfo_cursor_res[0]['MinTubeCurr']))
        XRayTubeCurrent.set('Avg',str(LDCTInfo_cursor_res[0]['AvgTubeCurr']))
        ExposuremAs = ET.SubElement(LDCTInfo, 'ExposuremAs')
        ExposuremAs.set('Total',str(LDCTInfo_cursor_res[0]['TotalmAs']))
        ExposuremAs.set('Max',str(LDCTInfo_cursor_res[0]['MaxmAs']))
        ExposuremAs.set('Min',str(LDCTInfo_cursor_res[0]['MinmAs']))
        ExposuremAs.set('Avg',str(LDCTInfo_cursor_res[0]['AvgmAs']))
        ConvolutionKernel = ET.SubElement(LDCTInfo, 'ConvolutionKernel')
        ConvolutionKernel.text = str(LDCTInfo_cursor_res[0]['ConvKernel'])
        SingleCollimationWidth = ET.SubElement(LDCTInfo, 'SingleCollimationWidth')
        SingleCollimationWidth.text = str(LDCTInfo_cursor_res[0]['SingleCollWid'])
        TotalCollimationWidth = ET.SubElement(LDCTInfo, 'TotalCollimationWidth')
        TotalCollimationWidth.text = str(LDCTInfo_cursor_res[0]['TotalCollWid'])
        SeriesDescription = ET.SubElement(LDCTInfo, 'SeriesDescription')
        SeriesDescription.text = str(LDCTInfo_cursor_res[0]['SeriesDesc'])
        SliceThickness = ET.SubElement(LDCTInfo, 'SliceThickness')
        SliceThickness.text = str(LDCTInfo_cursor_res[0]['SliceThickness'])
        SpacingBtSlices = ET.SubElement(LDCTInfo, 'SpacingBtSlices')
        SpacingBtSlices.text = str(LDCTInfo_cursor_res[0]['SpacingBtSlices'])
        Gap = ET.SubElement(LDCTInfo, 'Gap')
        Gap.text = str(LDCTInfo_cursor_res[0]['Gap'])
        KVP = ET.SubElement(LDCTInfo, 'KVP')
        KVP.text = str(LDCTInfo_cursor_res[0]['KVP'])
        Pitch = ET.SubElement(LDCTInfo, 'Pitch')
        Pitch.text = str(LDCTInfo_cursor_res[0]['Pitch'])
        res = Annotation_res
        for row in res:
            if row['LabelName']=='OtherFinding':
                    OtherFinding = ET.SubElement(Main, 'Abnormality')
                    OtherFinding.set('X_Pos',str(row['x']))
                    OtherFinding.set('Y_Pos',str(row['y']))
                    OtherFinding.set('Z_Pos',str(row['z']))
                    OtherFinding.set('ROISize','50')
                    OtherFinding.set('Type','OtherFinding')
            else:
                    Abnormality = ET.SubElement(Main, 'Abnormality')
                    Abnormality.set('X_Pos',str(row['x']))
                    Abnormality.set('Y_Pos',str(row['y']))
                    Abnormality.set('Z_Pos',str(row['z']))
                    Abnormality.set('ROISize','50')
                    Abnormality.set('Type','Abnormality')
        trees = ET.ElementTree(Main)
        filename = 'Annotation.xml'
        trees.write(os.path.join(Dir,filename)) 